# 3. 應用 LoRA 進行模型微調

在本筆記本中，我們將進入最核心的環節：使用 `peft` 函式庫將 LoRA (Low-Rank Adaptation) 應用到我們已經載入的量化模型上，並設定訓練流程來進行微調。

## 步驟 1: 匯入必要的函式庫

首先，我們匯入 `peft` 函式庫中用於設定 LoRA 的 `LoraConfig` 和 `get_peft_model` 函式，以及 `transformers` 中用於設定訓練參數的 `TrainingArguments` 和執行訓練的 `Trainer`。


In [1]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import TrainingArguments, Trainer
import torch

# 重新執行上一筆記本的程式碼，確保模型和 Tokenizer 已載入
# 在實際操作中，您也可以將模型和 Tokenizer 儲存後再載入，或使用其他方式在筆記本間共享變數
# %run ./02-load-model-and-dataset.ipynb


## 載入模型

In [2]:
# 模型下載後會自動快取在 Hugging Face 的 default cache 目錄，預設（對於大多數作業系統）在：
#   ~/.cache/huggingface/hub
# 你可以查詢實際快取目錄：
from huggingface_hub import hf_hub_download
from transformers.utils.hub import TRANSFORMERS_CACHE
import os

# 修正 ImportError：新版 huggingface_hub 裡 "hf_hub_cache_dir" 可能已被移除或變動
# 只保留正確可用的 import，如果需要取得快取目錄，建議直接使用 TRANSFORMERS_CACHE 或 os.path.expanduser

# from huggingface_hub import hf_hub_download  # 如果要用 hf_hub_download 可以保留，否則可註解


print(f"Hugging Face 預設 model cache 路徑：{os.path.expanduser('~/.cache/huggingface/hub')}")
print(f"Transformers 預設快取目錄（可由環境變數 TRANSFORMERS_CACHE 控制）: {TRANSFORMERS_CACHE}")


Hugging Face 預設 model cache 路徑：/home/os-sunnie.gd.weng/.cache/huggingface/hub
Transformers 預設快取目錄（可由環境變數 TRANSFORMERS_CACHE 控制）: /home/os-sunnie.gd.weng/.cache/huggingface/hub


In [3]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "meta-llama/Llama-2-7b-chat-hf"

# 設定量化參數
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 載入 Tokenizer
# 注意：你需要先到 Hugging Face 網站申請 Llama-2 的使用權限，並使用 `huggingface-cli login` 登入
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token # 設定 pad_token

# 載入量化後的模型
# 使用 device_map={"": 0} 明確指定將整個模型載入到 GPU 0
# 這是解決量化訓練 TypeError 的關鍵
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map={"": 0},
)

print("模型與 Tokenizer 載入成功！")
print(model)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

模型與 Tokenizer 載入成功！
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )


In [4]:
from datasets import load_dataset

dataset_name = "mlabonne/guanaco-llama2-1k"
dataset = load_dataset(dataset_name, split="train")

print("資料集載入成功！")
print(dataset)
print("範例資料：")
print(dataset[0])


# 檢查資料集的欄位名稱，並依據欄位動態格式化
def format_prompt(sample):
    # 如果資料集中有 'instruction' 欄位（如 Alpaca 格式）
    if "instruction" in sample:
        instruction = sample.get('instruction', '')
        input_text = sample.get('input', '')
        output = sample.get('output', '')

        if input_text and input_text.strip():
            prompt = f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n{output}"
        else:
            prompt = f"### Instruction:\n{instruction}\n\n### Response:\n{output}"
        return prompt
    # 如果資料集中只有 'text' 欄位（如 Guanaco 格式）
    elif "text" in sample:
        text = sample.get('text', '')
        # 嘗試剖析「Instruction」、「Input」、「Response」區塊，若不存在則直接回傳原始 text
        # Guanaco 格式的 text 原始樣貌通常為 <s>[INST]...[/INST] ...</s>
        # 這裡可以直接回傳 text 或視需要進行 strip
        return text.strip()
    else:
        return "擷取失敗"

# 測試格式化結果
print("格式化後的範例資料：")
print(format_prompt(dataset[0]))



資料集載入成功！
Dataset({
    features: ['text'],
    num_rows: 1000
})
範例資料：
{'text': '<s>[INST] Me gradué hace poco de la carrera de medicina ¿Me podrías aconsejar para conseguir rápidamente un puesto de trabajo? [/INST] Esto vale tanto para médicos como para cualquier otra profesión tras finalizar los estudios aniversarios y mi consejo sería preguntar a cuántas personas haya conocido mejor. En este caso, mi primera opción sería hablar con otros profesionales médicos, echar currículos en hospitales y cualquier centro de salud. En paralelo, trabajaría por mejorar mi marca personal como médico mediante un blog o formas digitales de comunicación como los vídeos. Y, para mejorar las posibilidades de encontrar trabajo, también participaría en congresos y encuentros para conseguir más contactos. Y, además de todo lo anterior, seguiría estudiando para presentarme a las oposiciones y ejercer la medicina en el sector público de mi país. </s>'}
格式化後的範例資料：
<s>[INST] Me gradué hace poco de la carrera d

In [5]:
# 如果你想計算一段文字經 tokenizer 後有多少個 token，可以這樣做：
sample_text = format_prompt(dataset[0])
tokenized = tokenizer(sample_text, return_tensors=None)
num_tokens = len(tokenized["input_ids"])
print(f"該樣本的 token 數量為: {num_tokens}")

# 也可計算整個 dataset 每筆資料的 token 數量
token_counts = [len(tokenizer(format_prompt(sample), return_tensors=None)["input_ids"]) for sample in dataset]
print(f"所有樣本的最大 token 數量: {max(token_counts)}")
print(f"所有樣本的平均 token 數量: {sum(token_counts)/len(token_counts)}")


該樣本的 token 數量為: 218
所有樣本的最大 token 數量: 7833
所有樣本的平均 token 數量: 462.907


In [6]:
# 對資料集中的每個樣本應用格式化函數，並進行 tokenization
# max_length 可以根據您的 GPU 記憶體進行調整
# remove_columns=['text'] 會移除原始的 text 欄位，只保留 tokenizer 輸出的 input_ids, attention_mask 等
dataset = dataset.map(lambda samples: tokenizer(format_prompt(samples), max_length=512, truncation=True), remove_columns=['text'])


In [7]:
dataset

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 1000
})

## 步驟 2: 設定 LoRA (LoraConfig)

為了將 LoRA 應用到我們的模型上，我們需要建立一個 `LoraConfig` 物件。這個物件會告訴 `peft` 函式庫如何設定 LoRA adapter。

關鍵參數說明：
- `r`: LoRA 的秩 (rank)。這是一個關鍵超參數，決定了低秩矩陣的大小。較小的 `r` 意味著更少的參數和更快的訓練，但可能會犧牲一些性能。常見的設定值為 8, 16, 32, 64。
- `lora_alpha`: LoRA 的縮放因子，可以理解為學習率的縮放。公式為 `alpha / r`。通常設定為 `r` 的兩倍。
- `target_modules`: 指定要應用 LoRA 的模組名稱。對於 Transformer 模型，通常是注意力機制中的查詢 (query) 和值 (value) 投影層，即 `"q_proj"` 和 `"v_proj"`。
- `lora_dropout`: 在 LoRA 層中使用的 dropout 比率。
- `bias`: 設定 bias 是否可訓練。`"none"` 表示所有 bias 都不訓練。
- `task_type`: 指定任務類型。對於語言模型，我們設定為 `"CAUSAL_LM"`。

設定好 `LoraConfig` 後，我們使用 `get_peft_model` 函式將其應用到我們的基礎模型上。該函式會凍結所有原始模型參數，並在指定模組上加上 LoRA adapter。


In [8]:
# 建立 LoraConfig
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# 將 LoRA 應用到模型上
peft_model = get_peft_model(model, lora_config)

# 打印出模型中可訓練的參數
peft_model.print_trainable_parameters()


trainable params: 8,388,608 || all params: 6,746,804,224 || trainable%: 0.1243


## 步驟 3: 設定訓練參數與啟動訓練

現在我們的模型已經準備好進行微調，最後一步是設定訓練流程。我們使用 Hugging Face `Trainer` 來處理大部分的訓練細節。

首先，建立一個 `TrainingArguments` 物件來定義訓練的各項參數：
- `output_dir`: 訓練過程中模型 checkpoint 和日誌的儲存目錄。
- `per_device_train_batch_size`: 每個 GPU 上的訓練批次大小。
- `gradient_accumulation_steps`: 梯度累積步數，用於在記憶體不足時模擬更大的批次大小。
- `learning_rate`: 學習率。
- `num_train_epochs`: 訓練的總輪數。
- `logging_steps`: 每隔多少步記錄一次訓練日誌。
- `fp16`: 啟用混合精度訓練以加速並節省記憶體。

接著，我們實例化一個 `Trainer`，並傳入模型、訓練參數、資料集等。最後，呼叫 `trainer.train()` 即可開始微調！

**注意**: 即使是 PEFT，微調 7B 模型仍然需要一些時間。在 Colab 的 T4 GPU 上，這個步驟大約需要 15-20 分鐘。


In [9]:

# 準備模型進行 k-bit 訓練並加上 LoRA 配置
model.gradient_checkpointing_enable()
peft_model = prepare_model_for_kbit_training(model)
peft_model = get_peft_model(peft_model, lora_config)

# 設定訓練參數
training_args = TrainingArguments(
    output_dir="./lora-llama2-7b-chat",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=12,
    learning_rate=2e-4,
    num_train_epochs=1,
    logging_steps=30,
    bf16=True,
    save_total_limit=2,
    overwrite_output_dir=True,
)

# 建立 Trainer
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=dataset,
    # 我們需要提供一個 data_collator 來將資料整理成批次
    # 對於語言模型，我們通常使用 DataCollatorForLanguageModeling
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

# 關閉 cache 以解決一個已知的梯度檢查點問題
peft_model.config.use_cache = False

# 開始訓練
trainer.train()

print("訓練完成！")


# 進度條格式 [12/21 09:26 < 08:30, ...] 的意思如下：
# - 12/21 代表目前訓練 12 步 (steps)，總共需訓練 21 步。
#   這個總步數（例如 21）是由 Trainer 根據：
#   總樣本數、訓練 batch size (per_device_train_batch_size)、gradient_accumulation_steps、
#   與總 epoch 數 (num_train_epochs) 計算而來。
#
# 算法大致如下：
#     總步數 = ceil(樣本數 / (每卡batch_size * 卡數) / gradient_accumulation_steps) * 總 epoch 數
#
# 以本例來說，如果你的 dataset 長度為 N 條，單卡 batch size = 4，
# 假設沒用多卡（單張GPU），gradient_accumulation_steps=12，
# 1 epoch 訓練 iteration 數量 = ceil(N / 4 / 12)
# 設 num_train_epochs = 1, 則 total steps = ceil(N / 4 / 12) * 1
# 21 就是這個 total steps 的值。
#
# 進度條會在每一步更新左側「已完成/總步數」狀態。




/home/os-sunnie.gd.weng/python_workstation/side-project/iSpan_LLM-One-Piece/00-Course_Setup/.venv/lib/python3.10/site-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/home/os-sunnie.gd.weng/python_workstation/side-project/iSpan_LLM-One-Piece/00-Course_Setup/.venv/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


[2025-10-21 15:34:54,618] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/os-sunnie.gd.weng/python_workstation/side-project/iSpan_LLM-One-Piece/00-Course_Setup/.venv/lib/python3.10/site-packages/deepspeed/runtime/zero/linear.py:49: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, weight, bias=None):
/home/os-sunnie.gd.weng/python_workstation/side-project/iSpan_LLM-One-Piece/00-Course_Setup/.venv/lib/python3.10/site-packages/deepspeed/runtime/zero/linear.py:67: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):
/home/os-sunnie.gd.weng/python_workstation/side-project/iSpan_LLM-One-Piece/00-Course_Setup/.venv/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant i

Step,Training Loss


訓練完成！


模型微調已經完成！在下一個筆記本 `04-inference-and-evaluation.ipynb` 中，我們將學習如何使用這個微調後的模型進行推理。
